# Global Diferential Privacy Assessement: Multivariate case for Neuroimaging data

In this notebook, we will explore further the Multivariate case of the Global Differential Privacy for Neuroimaging data. After running this notebook, you will be able to:
- Understand the concept and applying Gaussian based Global Differential Privacy for Multivaritate case.


### questions:
#### 1. what can i consider user_output?
#### 2. how should I approach dimensionality? cuz it takes considerably long time given a test file of size (70 * 70 * 50, 248)

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, wget, shutil, math
from math import sqrt
from scipy.stats import pearsonr
from itertools import combinations
from rich import print
import warnings, random
warnings.filterwarnings("ignore")
#from scipy.stats import multivariate_normal
from numpy.random import multivariate_normal


import os, sys, subprocess, csv, random, wget, shutil
import numpy as np
from IPython.display import HTML, display
import matplotlib.pyplot as plt


In [53]:
script = os.path.join(os.path.dirname(os.getcwd())+"/CalibrateNoise/", "main.py") 
file = "input/anat/original/"
clean = True

os.makedirs("img", exist_ok=True)
link_original = "https://s3.amazonaws.com/openneuro.org/ds004934/sub-SAXNES2s001/func/sub-SAXNES2s001_task-DOTS_run-001_bold.nii.gz?versionId=R0fwRS9fxw8CcPZnb4zYsw9I5v19aAbP"
wget.download(link_original)
os.makedirs(file, exist_ok=True)
file_ = [os.path.join(root, file) for root, _, files in os.walk(os.getcwd()) for file in files if file.endswith(".gz")]
shutil.copy2(file_[0], file)
os.remove(file_[0])
print(f"\nOriginal file downloaded.")

result = subprocess.run(
    ["python3", script, file],
    capture_output=True,
    text=True
)
print(result.stdout)

if clean:
    ! rm -r input
    print(f"Cleaning up finished")    


100% [....................................................] 60545196 / 60545196

Original file downloaded.

(70, 70, 50, 248)

Total time taken: 00:00:02

Cleaning up finished

In [55]:
np.random.seed(42)

size = [3,5]
test = np.random.normal(loc=0.0, scale=1.0, size=size)
print("Test data: \n", test)

def user_output(data):
    return np.array([np.mean(data, axis=0)])
pipeline_output = user_output(test)
print("User output: \n", pipeline_output)
off_d = np.cov([test[:,i] for i in range(test.shape[1])])
print("off diagonal: \n", off_d)

loo_output = np.array([np.mean(np.delete(test,i,axis=0), 0) for i in range(test.shape[0])])
loo_scale = np.std(loo_output, axis=0)
sensitivity = np.max(np.abs(loo_output - pipeline_output), 0)
cov = np.diag((2 * loo_scale)**2) 
print("Cov: \n", cov)

combined = np.abs(np.array(off_d))   
np.fill_diagonal(combined, np.diag(cov))

print("Combined cov: \n", combined)

while True:
    noise = multivariate_normal(mean=np.zeros(test.shape[1]), cov=combined)#.rvs()
    if np.all(noise >= sensitivity):
        break
noisy_output = pipeline_output + noise

print("Noisy output: \n",noisy_output)

Test data: 
 [[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337]
 [-0.23413696  1.57921282  0.76743473 -0.46947439  0.54256004]
 [-0.46341769 -0.46572975  0.24196227 -1.91328024 -1.72491783]]

User output: 
 [[-0.06694683  0.32507292  0.55236185 -0.28657492 -0.47217039]]

off diagonal: 
 [[ 0.25142769 -0.07865735  0.07041913  0.84776193  0.23059293]
 [-0.07865735  1.20645856  0.23551395  0.10928208  1.0765039 ]
 [ 0.07041913  0.23551395  0.07584571  0.31904778  0.31489132]
 [ 0.84776193  0.10928208  0.31904778  2.97714594  1.141487  ]
 [ 0.23059293  1.0765039   0.31489132  1.141487    1.32785305]]

Cov: 
 [[0.16761846 0.         0.         0.         0.        ]
 [0.         0.80430571 0.         0.         0.        ]
 [0.         0.         0.05056381 0.         0.        ]
 [0.         0.         0.         1.98476396 0.        ]
 [0.         0.         0.         0.         0.88523537]]

Combined cov: 
 [[0.16761846 0.07865735 0.07041913 0.84776193 0.23059293]
 [0.07865735 0.80430571 0.23551395 0.10928208 1.0765039 ]
 [0.07041913 0.23551395 0.05056381 0.31904778 0.31489132]
 [0.84776193 0.10928208 0.31904778 1.98476396 1.141487  ]
 [0.23059293 1.0765039  0.31489132 1.141487   0.88523537]]

Noisy output: 
 [[0.59438584 1.62300733 1.05479926 1.37089137 0.83360385]]

In [57]:
np.random.seed(42)

size = [70, 70, 50, 248]
test = np.random.normal(loc=0.0, scale=1.0, size=size)

def user_output(data):
    return np.array([np.mean(data, axis=0)])
pipeline_output = user_output(test)

off_d = np.cov([test[:,i] for i in range(test.shape[1])])

loo_output = np.array([np.mean(np.delete(test,i,axis=0), 0) for i in range(test.shape[0])])
loo_scale = np.std(loo_output, axis=0)
sensitivity = np.max(np.abs(loo_output - pipeline_output), 0)
cov = np.diag((2 * loo_scale)**2) 
combined = np.abs(np.array(off_d))   
np.fill_diagonal(combined, np.diag(cov))


while True:
    noise = multivariate_normal(mean=np.zeros(test.shape[1]), cov=combined)#.rvs()
    if np.all(noise >= sensitivity):
        break
noisy_output = pipeline_output + noise

print("Noisy output: \n",noisy_output)


ValueError: m has more than 2 dimensions

In [59]:
from sklearn.decomposition import PCA

size = [70, 70, 50, 248]
test = np.random.normal(loc=0.0, scale=1.0, size=size)

X = test.reshape(-1, 248).T 

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

print("shape:", X_pca.shape) 

shape:
(248, 2)

In [ ]:
def user_output(data):
    return np.array([np.mean(data, axis=0)])
pipeline_output = user_output(test)

off_d = np.cov([X[:,i] for i in range(X.shape[1])])

loo_output = np.array([np.mean(np.delete(X,i,axis=0), 0) for i in range(X.shape[0])])
loo_scale = np.std(loo_output, axis=0)
sensitivity = np.max(np.abs(loo_output - pipeline_output), 0)
cov = np.diag((2 * loo_scale)**2) 
combined = np.abs(np.array(off_d))   
np.fill_diagonal(combined, np.diag(cov))


while True:
    noise = multivariate_normal(mean=np.zeros(test.shape[1]), cov=combined)#.rvs()
    if np.all(noise >= sensitivity):
        break
noisy_output = pipeline_output + noise

print("Noisy output: \n",noisy_output)

In [248]:


def user_output(data):
    return np.array([np.mean(data, axis=0)])

def dp_2d(data, pipeline_output):

    off_d = np.cov([data[:,0],data[:,1]])
    loo_output = np.array([np.mean(np.delete(data,i,axis=0), 0) for i in range(data.shape[0])])
    loo_scale = np.std(loo_output, axis=0)
    sensitivity = np.max(np.abs(loo_output - pipeline_output), 0)
    cov = np.diag((2 * loo_scale)**2) 

    combined = np.abs(np.array(off_d))   
    np.fill_diagonal(combined, np.diag(cov)) 
    while True:
        noise = multivariate_normal(mean=[0, 0], cov=combined)#.rvs()
        if np.all(noise >= sensitivity):
            break
    noisy_output = pipeline_output + noise
    
    return noisy_output, sensitivity

def iqr_bounds(data, axis=0):

    data = np.asarray(data)
    q1 = np.percentile(data, 25, axis=axis)
    q3 = np.percentile(data, 75, axis=axis)
    
    iqr = q3 - q1
    lower_bounds = q1 - 1.5 * iqr
    upper_bounds = q3 + 1.5 * iqr
    
    return iqr, lower_bounds, upper_bounds

def outlier_(value, lower_bounds, upper_bounds):

    value = np.asarray(value)
    lower_bounds = np.asarray(lower_bounds)
    upper_bounds = np.asarray(upper_bounds)
    
    return (value < lower_bounds) | (value > upper_bounds)

